In [1]:
cd /content/drive/MyDrive/2022-1/캡스톤/comebackhome

/content/drive/MyDrive/2022-1/캡스톤/comebackhome


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm, tqdm_notebook

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#kobert
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ts9i8xy5
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ts9i8xy5


In [5]:
#kobert tokenizer
! pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-2bufwfam/kobert-tokenizer_560ff2b4a42f404a8dd33cfc526ff577
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-2bufwfam/kobert-tokenizer_560ff2b4a42f404a8dd33cfc526ff577


------------------------------

# 1. 데이터 로드

In [6]:
X_train = pd.read_csv("X_train_trans.csv")
y_train = pd.read_csv("y_train_trans.csv")['level5/diagnosis']
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")['level5/diagnosis']

# 2. 여러개의 컬럼 -> 하나의 문자열로 변한

In [7]:
X_test = X_test.fillna("")
X_test['All'] = (X_test['Chief complaint'].map(str) +'. '+ X_test['Age'].map(str) + '. '+
                X_test['Sex'].map(str) + '. '+
                X_test['Onset'].map(str) + '. '+ X_test['Location'].map(str) + '. ' +
                X_test['Duration'].map(str) + '. ' + X_test['Course'].map(str) + '. ' +
                X_test['Experience'].map(str) + '. ' + X_test['Character'].map(str) + '. ' +
                X_test['Associated Sx.'].map(str)+'. '+ X_test['Factor'].map(str) + '. ' +
                X_test['Event'].map(str) + '. ' + X_test['약물 투약력'].map(str) + '. ' +
                X_test['사회력'].map(str) + '. ' + X_test['가족력'].map(str) + '. ' +
                X_test['외상력'].map(str) + '. ' + X_test['과거력'].map(str) + '. ' +
                X_test['여성력'].map(str) + '. ' + X_test['Obesity']).apply(lambda row: row.strip())

In [8]:
X_train = X_train.fillna("")
X_train['All'] = (X_train['Chief complaint'].map(str) +'. '+ X_train['Age'].map(str) + '. '+
                X_train['Sex'].map(str) + '. '+
                X_train['Onset'].map(str) + '. '+ X_train['Location'].map(str) + '. ' +
                X_train['Duration'].map(str) + '. ' + X_train['Course'].map(str) + '. ' +
                X_train['Experience'].map(str) + '. ' + X_train['Character'].map(str) + '. ' +
                X_train['Associated Sx.'].map(str)+'. '+ X_train['Factor'].map(str) + '. ' +
                X_train['Event'].map(str) + '. ' + X_train['약물 투약력'].map(str) + '. ' +
                X_train['사회력'].map(str) + '. ' + X_train['가족력'].map(str) + '. ' +
                X_train['외상력'].map(str) + '. ' + X_train['과거력'].map(str) + '. ' +
                X_train['여성력'].map(str) + '. ' + X_train['Obesity']).apply(lambda row: row.strip())


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)
y_train

array([ 87, 237, 144, ..., 183,  16, 104])

# 3. 학습

#### ver5) KoBert + 번역데이터 


In [10]:
MODEL = 'skt/kobert-base-v1'
max_len = 128
batch_size = 16
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5 
device = torch.device('cuda:0')
dr_rate = 0.2

In [12]:
from kobert import get_pytorch_kobert_model
from kobert import get_tokenizer
from kobert_tokenizer import KoBERTTokenizer
import gluonnlp as nlp

kobert_model, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/2022-1/캡스톤/comebackhome/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/2022-1/캡스톤/comebackhome/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/drive/MyDrive/2022-1/캡스톤/comebackhome/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


##### 데이터 임베딩 & 데이터 로더

In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [11]:
dataset_train = [[x,y] for x, y in zip(X_train['All'], y_train)]
dataset_test = [[x,y] for x, y in zip(X_test['All'], y_test)]

In [14]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


##### 모델 정의

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=len(le.classes_),
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [16]:
model = BERTClassifier(kobert_model,  dr_rate=dr_rate).to(device)

##### 모델 학습

In [17]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = optim.AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [18]:
from transformers.optimization import get_cosine_schedule_with_warmup
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [19]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

### top3 추론 

In [ ]:
with torch.no_grad():
    model.eval()
    output = []
    top3 = []
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        output.extend(out.argmax(dim = 1).detach().cpu().numpy())
        top3.extend(torch.topk(out, 3, dim = 1).indices.detach().cpu().tolist())
        test_acc += calc_accuracy(out, label)

# 3. 모델 저장

#### save on GPU, load on CPU

In [ ]:
mkdir server_ver2

In [ ]:
np.save('./server_ver2/classes.npy', le.classes_)
torch.save(model.state_dict(), './server_ver2/kobert2_state_dict.pth')